In [21]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import sys
sys.path.append('../..') # Adiciona o diretório superior ao caminho de importação para acessar funções utilitárias
from functions.db_inserts import * # Importa funções para inserção de dados no banco de dados
from functions.aws_functions import * # Importa funções relacionadas à AWS
from functions.select_cols_functions import * # Importa funções para seleção e processamento de colunas
from sklearn.metrics import mean_absolute_error, cohen_kappa_score

In [23]:
env_mode = 'prd' # Define o modo de ambiente: 'dev' para desenvolvimento, 'prd' para produção

### Lendo arquivo fonte

In [24]:
bucket_name = "adoptai-trusted-prod-tcc" # Nome do bucket S3 de onde os dados serão carregados

In [25]:
if env_mode == 'dev':
    # Modo de desenvolvimento: carrega e processa os dados localmente
    source = 'feature_engineering'
    df_trusted = pd.read_csv(f'../../../data/{source}.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = [f'{source}.csv', df_trusted]
elif env_mode == 'prd':
    # Modo de produção: carrega e processa os dados a partir do S3
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Utilizando LabelEncoded nas colunas categóricas

In [26]:
df_trusted = adjust_categoric_values(df_trusted)

### Parâmetros de gravação

In [27]:
nome_fonte = s3[0] # Armazena o nome do arquivo de dados utilizado
nome_modelo = 'Decision Tree (Entropy)' # Define o nome do modelo
parameters = {
    'max_depth': [5, 10, 20, 30], # Define os parâmetros de profundidade máxima para a árvore de decisão
}

### Preparativos para executar o modelo

In [28]:
# Separar as features (x) e os rótulos (y)
x = df_trusted.iloc[:, :-1] # Seleciona todas as colunas, exceto a última, como features
y = df_trusted.iloc[:, -1] # Seleciona a última coluna como rótulo

In [29]:
# Dividir os dados em conjuntos de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

###  Rodando modelo Decision Tree

In [31]:
# Listas para armazenar métricas de desempenho e tempos de execução
accuracy = []
precision = []
recall = []
f1 = []
mae = []
qwk = []

start_times = []
end_times = []

In [32]:
# Treina o modelo usando diferentes profundidades máximas para a árvore de decisão
for max_depth in parameters['max_depth']:

    actual_params = {
        'criterion': 'entropy', # Define o critério de divisão como entropia
        'max_depth': max_depth, # Define a profundidade máxima atual
        'min_samples_split': 2, # Define o número mínimo de amostras para dividir um nó
        'min_samples_leaf': 2 # Define o número mínimo de amostras em uma folha
    }

    start_times.append(time.time()) # Registra o tempo de início
    model = DecisionTreeClassifier(**actual_params, random_state=42) # Inicializa o modelo com os parâmetros atuais
    model.fit(x_train, y_train) # Treina o modelo com os dados de treinamento

    # Fazer previsões no conjunto de teste
    y_pred = model.predict(x_test)

    # Avaliar o modelo e armazenar as métricas
    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average=None))
    recall.append(recall_score(y_test, y_pred, average=None))
    f1.append(f1_score(y_test, y_pred, average=None))
    mae.append(mean_absolute_error(y_test, y_pred))
    qwk.append(cohen_kappa_score(y_test, y_pred, weights='quadratic'))

    end_times.append(time.time()) # Registra o tempo de término

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Obtendo métricas de desempenho

In [33]:
# Armazena as métricas calculadas
metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

### Gravando dados no banco

In [34]:
# Prepara os dados para inserção no banco de dados
model = {'nome_modelo': nome_modelo, 'nome_fonte': nome_fonte}
execution = {
    'accuracy': accuracy, 
    'mae': mae,
    'qwk': qwk,
    'start_time': start_times, 
    'end_time': end_times
    }
hyperparams = parameters
desempenho = {
    'precision_values': metrics['precision'], 
    'recall_values': metrics['recall'], 
    'f1_values': metrics['f1_score']
    }

In [36]:
# Insere os valores calculados nas tabelas do banco de dados
insert_values(model, execution, hyperparams, desempenho, env=env_mode)